In [5]:
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader("The-ECG-Made-Easy-9th-Edition.pdf")
docs = loader.load()

In [6]:
print("Number of pages in the PDF:",len(docs))
docs[2].page_content


Number of pages in the PDF: 269


"Table of Contents\nCover image\nTitle Page\nCopyright\nHow to use this book\nPart 1: The ECG made very easy indeed: a beginner's guide\nPart 2: The basics: the fundamentals of ECG recording, reporting\nand interpretation\nPart 3: Making the most of the ECG: the clinical interpretation of\nindividual ECGs\nPart 4: Now test yourself\nQuick reminders\nFurther reading\nPreface\nPart 1 The ECG made very easy indeed: a beginner's\nguide\n"

In [17]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings

text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

/var/folders/8c/zybyw_md6p1c6nb6rz716x5h0000gn/T/ipykernel_13490/2762011161.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
/Users/sanjaibalajee/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [8]:
for i in range(len(documents)):
    print()
    print(f"CHUNK : {i+1}")
    print(documents[i].page_content)

NameError: name 'documents' is not defined

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()

# Create the vector store 
vector = FAISS.from_documents(documents, embedder)

/var/folders/8c/zybyw_md6p1c6nb6rz716x5h0000gn/T/ipykernel_13490/2495835858.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings()
/var/folders/8c/zybyw_md6p1c6nb6rz716x5h0000gn/T/ipykernel_13490/2495835858.py:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [6]:
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke("Normal ECG patterns")

In [10]:
retrieved_docs

NameError: name 'retrieved_docs' is not defined

In [11]:
from langchain_community.llms import Ollama
llm = Ollama(model="phi3")

/var/folders/8c/zybyw_md6p1c6nb6rz716x5h0000gn/T/ipykernel_13490/3849431076.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi3")


In [12]:
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

prompt = """
1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.\n
3. Keep the answer crisp and limited to 3,4 sentences.

Context: {context}

Question: {question}

Helpful Answer:"""


QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None,
              )

qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True,
              )

/var/folders/8c/zybyw_md6p1c6nb6rz716x5h0000gn/T/ipykernel_13490/622636874.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
/var/folders/8c/zybyw_md6p1c6nb6rz716x5h0000gn/T/ipykernel_13490/622636874.py:31: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  combine_documents_chain = StuffDocumentsChain(


NameError: name 'retriever' is not defined

In [12]:
print(qa("What are some features of normal ECG patterns?"))



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.

3. Keep the answer crisp and limited to 3,4 sentences.

Context: Context:
content:9
ECGs you must be able to
recognize
ECGs with clinical scenarios 164
ECG descriptions and interpretations 172
You should now be able to recognize the common ECG patterns, and
this final chapter contains twelve 12-lead ECGs for you to interpret. But do not forget two important things: first, an ECG comes from an
individual patient and must be interpreted with the patient in mind,
and second, there is little point in recording and interpreting an ECG
unless you are prepared to take some action based on your findings. This is a theme developed in the companion to this book, 150 ECG
Cases. The following ECGs (1–12) are in no particular 

### RAG EVALUATION 

In [13]:
!pip install ragas==0.0.22

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 6.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [20]:
from ragas.langchain.evalchain import RagasEvaluatorChain
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
)

ValidationError: 1 validation error for OpenAIEmbeddings
openai_api_key
  Extra inputs are not permitted [type=extra_forbidden, input_value='no-key', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden

In [19]:
!pip install ragas==0.0.22

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.1/387.1 KB 9.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.4/982.4 KB 11.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 KB 10.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 KB 10.0 MB/s eta 0:00:00
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 9.8 MB/s eta 0:00:00:00:010:01m
  Using cached pandas-2.2.3-cp310-cp310-macosx_11_0_arm64.whl (11.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 KB 6.4 MB/s eta 0:00:00
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
 